In [1]:
from __future__ import print_function

In [2]:
import sys
import subprocess
import json
sys.path.append('../build/')

In [3]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [4]:
import versor as vsr
import autodiff_multivector as ad
from motor_estimation import MotorEstimationSolver

# Automatic Multivector Differentiation

In [5]:
jac, statements = ad.diff_ceres(vsr.Vec(1,2,3), vsr.Biv(1,1,1))
print(jac)
print(statements)

[[-5.]
 [-2.]
 [ 3.]]
[[-0. -1. -1.]
 [ 1.  0. -1.]
 [ 1.  1.  0.]]


In [6]:
jac, statements = ad.diff_adept(vsr.Vec(1,2,3), vsr.Biv(1,1,1))
print(jac)
print(statements)

[[ 0. -1. -1.]
 [ 1.  0. -1.]
 [ 1.  1.  0.]]
1: d[6] = 0
2: d[7] = 0
3: d[8] = 0
4: d[6] =  + -1*d[2] + -3*d[4] + -1*d[1] + -2*d[3]
5: d[7] =  + -1*d[2] + -3*d[5] + 1*d[0] + 1*d[3]
6: d[8] =  + 1*d[1] + 2*d[5] + 1*d[0] + 1*d[4]



In [7]:
ad.diff_ceres_rotor_matrix(pi/2, vsr.Vec(1,0,0))

[array([[ 0.],
        [ 0.],
        [ 0.]]), array([[ 0.],
        [ 0.],
        [ 0.]])]

In [8]:
res, jac, statements = ad.diff_adept_rotor_matrix(pi/2, vsr.Vec(1,0,0))

In [9]:
print(res)
print(jac)

[[ 0.]
 [ 0.]
 [ 0.]]
[[ 0.]
 [ 0.]
 [ 0.]]


In [10]:
res, jac = ad.diff_ceres_rotor_versor(0.5, vsr.Vec(1,0,0))
print(res)
print(jac)

[[ 0.8776]
 [ 0.4794]
 [ 0.    ]]
[[-0.4794]
 [ 0.8776]
 [ 0.    ]]


In [11]:
res, jac, statements = ad.diff_adept_rotor_versor(pi/2, vsr.Vec(1,0,0))
print(res)
print(jac)
# print(statements)

[[ 0.]
 [ 1.]
 [ 0.]]
[[-1.]
 [ 0.]
 [ 0.]]


In [12]:
vsr.Vec(1,0,0).spin(vsr.Rot(cos(0.25), -sin(0.25),0,0))

Vec: [ 0.88 0.48 0 ]

In [13]:
def run_benchmarks(name, repetitions):
    proc = subprocess.Popen(['/home/game/game/build/benchmark', 
                             '--benchmark_format=json', 
                             '--benchmark_filter={}'.format(name),
                             '--benchmark_repetitions={}'.format(repetitions)],
                            stdout=subprocess.PIPE)
    stdout = proc.communicate()[0]
    return json.loads(stdout)

benchmarks = run_benchmarks('BM_CeresRotor',1)
print(benchmarks['benchmarks'][0]['real_time'])

920


# Motor Estimation
## Performance

In [14]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(motor)

Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]


In [15]:
def benchmark_motor_estimation(n_points, num_linear_solver_threads, num_threads):
    sigma = 0.09
    points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() 
                for i in range(n_points)]
    points_b = [point.spin(motor) for point in points_a]
    points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                                + sigma * np.random.randn(3))).null() 
                      for point in points_b]
    initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    mes = MotorEstimationSolver(initial_motor)
    for a, b in zip(points_a, points_b_noisy):
        mes.add_point_correspondences_residual_block(a,b)
    mes.set_parameterization('BIVECTOR_GENERATOR')
    mes.linear_solver_type = 'DENSE_QR'
    mes.num_linear_solver_threads = num_linear_solver_threads
    mes.num_threads = num_threads
    (estimated_motor, summary, motors) = mes.solve()
#     print(summary['full_report'])
    return summary['total_time_in_seconds']

In [16]:
time_10_1_1 = [benchmark_motor_estimation(10, 1,1) for i in range(1000)]
print(np.mean(time_10_1_1))
print(np.std(time_10_1_1))

0.00050008401298
9.47762783549e-05


In [31]:
time_100_4_4 = [benchmark_motor_estimation(10000, 128,32) for i in range(100)]
print(np.mean(time_100_4_4))
print(np.std(time_100_4_4))

0.0476819423995
0.00166339272053


In [18]:
# n_points = [10, 100, 1000, 10000, 100000]
n_points = [10]
n_linear_solver_threads = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]
n_threads = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]

In [19]:
res = {}
for i in n_points:
    for j in n_linear_solver_threads:
        for k in n_threads:
            total_time_in_seconds = benchmark_motor_estimation(i, j, k)
            res['{}_{}_{}'.format(i,j,k)] = total_time_in_seconds

In [20]:
print(res)

{'10_32_8': 0.025593224010663107, '10_2_1024': 0.041082893003476784, '10_1024_16': 0.0011323950020596385, '10_32_16': 0.0008658270089654252, '10_16_2048': 0.08148157800314948, '10_512_4': 0.00047682800504844636, '10_8_256': 0.009256713005015627, '10_128_2048': 0.0783414440084016, '10_512_2': 0.0061107290093787014, '10_64_2': 0.004629538001609035, '10_64_1': 0.0005250029935268685, '10_16_128': 0.007412380000459962, '10_64_4': 0.021858391002751887, '10_1024_8': 0.0005058050010120496, '10_64_8': 0.007026721999864094, '10_1024_4': 0.02503840198914986, '10_1024_2': 0.007055437003145926, '10_1024_1': 0.0005112130020279437, '10_256_32': 0.0012756069918395951, '10_64_256': 0.009087282000109553, '10_1_128': 0.013435982997179963, '10_4_16': 0.0009957559959730133, '10_2048_1': 0.0005131849902682006, '10_2_2048': 0.08146405201114248, '10_16_1024': 0.03737701699719764, '10_512_1024': 0.03114715698757209, '10_8_1024': 0.03170252501149662, '10_16_32': 0.0015387700113933533, '10_1024_64': 0.0025022150

In [21]:
summary['jacobian_evaluation_solver_time_in_seconds']

NameError: name 'summary' is not defined